# 1. QAOA, with translation invariance. 

In this example, we look at how the QAOA ansatz works the simulate the ground state of the following TFIM: 

$$ H = -\sum_j \sigma^z_j \sigma^z_{j+1} -  g\sum_i \sigma^x_i $$

Let's first solve this exactly, using exact diagonalization. To this end, we refer to some old code given by Aaron Szasz. First, we solve this for a number of values of $g$.

**Energy spectrum**

```N```: the number of sites

```num_pts```: how many values of the parameter ```g``` to use, in the interval from 0 to 2

```show_lowest```: if the value is ```False``` all $2^N$ eigenvalues will be plotted.  If it is a positive integer $n$, the lowest $n$ eigenvalues will be shown (or $2^N$ if that is less than $n$).

```gs_at_0```: if ```False``` the actual energies will be plotted; if ```True```, for each $g$ the ground state energy will be subtracted from all eigenvalues, so the plot is of "relative energy" above the ground state.

```per_site```: if ```True```, all energy eigenvalues will be divided by ```N``` to give the intensive quantity energy/site 

I recommend exploring these different options

In [ ]:
# Parameters
N = 4
num_pts = 21
show_lowest = 5
gs_at_0 = True
per_site = False

import numpy as np
import matplotlib.pyplot as plt

Sx = np.array( [[0,1],[1,0]] )
Sz = np.array( [[1,0],[0,-1]] )
Id = np.array( [[1,0],[0,1]] )


def d2b(i,N):
    b = np.zeros(N, dtype = int)
    for j in range(N):
        b[j] = (i//2**(N-1-j))%2
    return b

def d2b_v2(i,N):
    b = np.array( [ (i//2**(N-1-j))%2 for j in range(N) ] )
    return b

def b2d(b):
    N = len(b)
    d = np.sum( [ b[i]*2**(N-1-i) for i in range(N) ] )
    return d


def X_v2(N):
    """
    Input: N is the number of sites in a 1D Ising model
    Output: Matrix for sum of Sx terms, NOT including the overall minus sign or the factor of g
    
    Computed using Column-by-column method
    """
    X = np.zeros( (2**N,2**N), dtype = int )
    for col in range(2**N):
        b = d2b_v2(col, N)
        for j in range(N):
            coeff = 1
            new_b = b.copy()
            new_b[j] = 1-new_b[j] # Flip the spin on site j #(new_b[j]+1)%2
            row = b2d(new_b)
            X[row, col] += coeff
    return X

def ZZ_v3(N):
    """
    Input: N is the number of sites in a 1D Ising model
    Output: Matrix for sum of Sz Sz terms, NOT including the overall minus sign
    
    Computed using knowledge that operator is diagonal
    """
    ZZ_diag = np.zeros(2**N, dtype = int)
    for col in range(2**N):
        b = d2b(col,N)
        val = 0
        for j in range(N-1):
            val += (-1)**(b[j] + b[j+1])
        ZZ_diag[col] = val
    ZZ = np.diag(ZZ_diag)
    return ZZ


# Generate X and ZZ matrices using the fastest methods
X = X_v2(N) 
ZZ = ZZ_v3(N)

# Function to get H from these stored matrices
def get_H(g):
    return -ZZ - g*X

# Values of g to use in calculating/plotting
gs = np.linspace(0,2,num_pts)

# Create an empty table to store the results:
energies = np.zeros( (2**N,num_pts) )
energies[:,:] = np.nan 

for g_index, g in enumerate(gs): 
    H = get_H(g)
    e, v = np.linalg.eigh(H) 
    inds = np.argsort(e)
    e = e[inds]
    v = v[:, inds]
    
    energies[:,g_index] = e
    
if per_site:
    energies /= N
if gs_at_0:
    gs_es = energies[0].copy()
    for i in range(2**N):
        energies[i] -= gs_es

f, a = plt.subplots()
max_ind = 2**N if not show_lowest else show_lowest
for row in range(max_ind): 
    a.scatter(gs, energies[row])
a.set_xlabel('g', fontsize = 16);
a.set_ylabel('E', fontsize = 16);

We're actually more interested in the wavefunction for small $N$. So let's modify this so that the input is $g$ and the output is the ground state energy and the corresponding wavefunction. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Parameters
N = 4 # number of qubits
g = 2 # value for g

Sx = np.array( [[0,1],[1,0]] )
Sz = np.array( [[1,0],[0,-1]] )
Id = np.array( [[1,0],[0,1]] )

def d2b(i,N):
    b = np.array( [ (i//2**(N-1-j))%2 for j in range(N) ] )
    return b

def b2d(b):
    N = len(b)
    d = np.sum( [ b[i]*2**(N-1-i) for i in range(N) ] )
    return d

def X_v2(N):
    X = np.zeros( (2**N,2**N), dtype = int )
    for col in range(2**N):
        b = d2b(col, N)
        for j in range(N):
            coeff = 1
            new_b = b.copy()
            new_b[j] = 1-new_b[j] # Flip the spin on site j #(new_b[j]+1)%2
            row = b2d(new_b)
            X[row, col] += coeff
    return X

def ZZ_v3(N):
    ZZ_diag = np.zeros(2**N, dtype = int)
    for col in range(2**N):
        b = d2b(col,N)
        val = 0
        for j in range(N-1):
            val += (-1)**(b[j] + b[j+1])
        ZZ_diag[col] = val
    ZZ = np.diag(ZZ_diag)
    return ZZ

X = X_v2(N) 
ZZ = ZZ_v3(N)

def get_H(g):
    return -ZZ - g*X

H = get_H(g) # get the Hamiltonian
e, v = np.linalg.eigh(H)  # get pairs of eigenvectors and eigenvalues
inds = np.argsort(e) # use this to sort the energy spectrum 
e = e[inds]
v = v[:, inds]

#print(e)

print('Ground state energy: ', e[0])
print('Ground state wavefunction: \n', v[0])

Okay, now let's try implementing the QAOA ansatz and do optimization to get to the same ground state. To this end, let's first learn how to use minimization in python.

In [ ]:
# Nelder-Mead method
# Unconstrained

import numpy as np
from scipy.optimize import minimize

def rosen(x):
    """The Rosenbrock function"""
    return sum(100.0*(x[1:]-x[:-1]**2.0)**2.0 + (1-x[:-1])**2.0)

x0 = np.array([1.3, 0.7, 0.8, 1.9, 1.2])
res = minimize(rosen, x0, method='nelder-mead',
               options={'xatol': 1e-8, 'disp': True})

print(res.x)

In [ ]:
# Trust-region method
# Constrained

from scipy.optimize import Bounds
bounds = Bounds([0, -0.5], [1.0, 2.0])

def rosen(x):
    """The Rosenbrock function"""
    return sum(100.0*(x[1:]-x[:-1]**2.0)**2.0 + (1-x[:-1])**2.0)

x0 = np.array([0.5, 0])
res = minimize(rosen, x0, method='trust-constr', 
               #jac=rosen_der,                    --> this is the jacobian, which we won't have
               #hess=rosen_hess,                  --> this the hessian, which we won't have
               #constraints=[linear_constraint, nonlinear_constraint],  --> we also don't have linear constraints
               options={'verbose': 1}, bounds=bounds)


print(res.x)

In [ ]:
# Global Minimum Optimzation

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy import optimize


def eggholder(x):
    return (-(x[1] + 47) * np.sin(np.sqrt(abs(x[0]/2 + (x[1]  + 47))))
         -x[0] * np.sin(np.sqrt(abs(x[0] - (x[1]  + 47)))))

bounds = [(-512, 512), (-512, 512)]


x = np.arange(-512, 513)
y = np.arange(-512, 513)
xgrid, ygrid = np.meshgrid(x, y)
xy = np.stack([xgrid, ygrid])

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.view_init(45, -45)
ax.plot_surface(xgrid, ygrid, eggholder(xy), cmap='terrain')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('eggholder(x, y)')
plt.show()


results = dict()
results['shgo'] = optimize.shgo(eggholder, bounds)
print(results['shgo'])





## QAOA with constant $g$
 
Okay, now let's construct the QAOA wavefunction ansatz. To this end, we will need to construct a number of quantum gates in Python. Recall that the QAOA ansatz looks like the following

$$ |{\vec{\gamma}, \vec{\beta}}\rangle = e^{-i\beta_p g\sum_j \sigma^x_j} e^{-i\gamma_p \sum_j \sigma^z_j \sigma^z_{j+1}} \dots  e^{-i\beta_1 g\sum_j \sigma^x_j} e^{-i\gamma_1 \sum_j \sigma^z_j \sigma^z_{j+1}} |{++}\rangle $$

with $\vec{\gamma} = \{ \gamma_1,\dots,\gamma_p \}$ and $\vec{\beta} = \{ \beta_1,\dots,\beta_p \}$. It is found in SciPost paper by Tim Hsieh et al that $p = L/2 = 4/2 = 2$ is sufficient to target the ground state of this model. This means there is a total of four parameters $\{ \gamma_1,\gamma_2,\beta_1,\beta_2 \}$ in the ansatz. 

To construct the ansatz, we need to be able to construct the initial state $| ++ \rangle$, then the matrices. Once that is done, we will target the ground state in two ways: 

- We can try to maximize the overlap $|\psi | (\gamma,\eta)|^2$ between the wavefunctions.
- Or we can try to mimize the energy cost function: $\langle (\gamma,\beta) | H | (\gamma,\beta) \rangle $.

We will use both approaches and see whether we will get to the same state. 

In [ ]:
# QAOA for constant g
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import expm
import math as m
from scipy.optimize import Bounds, minimize


# Parameters
N = 4 # number of qubits
# g = 2*np.random.random_sample()  # randomly pick a g between 0 and 2
g = 2

print_Hamiltonian = False
print_Energy_Spectrum = False
print_Matrix_of_Eigenstates = False
print_Ground_State_Energy = True
print_Ground_State_Wfn = False
print_Optimized_Angles_Overlap = True
print_Optimized_State_Overlap = False
print_Overlap = True
print_Energy_Overlap = True
print_g = False
print_Energy_CostFunction = True
print_Optimized_Angles_CostFn = True
print_Optimized_State_CostFn = False


Sx = np.array( [[0,1],[1,0]] )
Sz = np.array( [[1,0],[0,-1]] )
Id = np.array( [[1,0],[0,1]] )

def d2b(i,N):
    b = np.array( [ (i//2**(N-1-j))%2 for j in range(N) ] )
    return b
def b2d(b):
    N = len(b)
    d = np.sum( [ b[i]*2**(N-1-i) for i in range(N) ] )
    return d

def X_v2(N):
    X = np.zeros( (2**N,2**N), dtype = int )
    for col in range(2**N):
        b = d2b(col, N)
        for j in range(N):
            coeff = 1
            new_b = b.copy()
            new_b[j] = 1-new_b[j] # Flip the spin on site j #(new_b[j]+1)%2
            row = b2d(new_b)
            X[row, col] += coeff
    return X

def ZZ_v3(N):
    ZZ_diag = np.zeros(2**N, dtype = int)
    for col in range(2**N):
        b = d2b(col,N)
        val = 0
        for j in range(N-1):
            val += (-1)**(b[j] + b[j+1])
        ZZ_diag[col] = val
    ZZ = np.diag(ZZ_diag)
    return ZZ

X = X_v2(N) 
ZZ = ZZ_v3(N)

def get_H(g):
    return -ZZ - g*X

H = get_H(g) # get the Hamiltonian
e, v = np.linalg.eigh(H)  # get pairs of eigenvectors and eigenvalues
inds = np.argsort(e) # use this to sort the energy spectrum 
e = e[inds]
v = v[:, inds]

ground_state = v.T[0]/np.linalg.norm(v.T[0])
ground_state_energy = e[0]

def ExpGB(beta,gamma):
    return expm(-(1j)*gamma*g*X)@expm(-(1j)*beta*ZZ)

def Plus():
    state = np.array([[1,1]])    
    for i in range(N-1):
        state = np.kron(np.array([[1,1]]), state)
    return state.T

def QAOA_state(params):
    # beta is an array of p = N/2 parameters
    # gamma is an array of p = N/2 parameters
    
    if len(params)%2 != 0: # if the number of parameters is not even
        print('The number of parameters must be even!')
        return 
    
    # initialize state in +
    QAOA_state = Plus()/np.linalg.norm(Plus())
    
    # split list of params
    beta  = params[:len(params)//2]
    gamma = params[len(params)//2:]
    
    # create QAOA ansatz
    for i in range(len(beta)):
        QAOA_state = np.dot(ExpGB(beta[i],gamma[i]), QAOA_state)
        
    return QAOA_state
    

def QAOA_overlap(params):
    # beta is an array of p = N/2 parameters
    # gamma is an array of p = N/2 parameters
        
    state = QAOA_state(params)
    # compute overlap
    # note: should return a negative --> optimization finds minima
    overlap = -np.abs( np.vdot(ground_state, state) )**2

    return overlap

def QAOA_cost_function(params):
    # beta is an array of p = N/2 parameters
    # gamma is an array of p = N/2 parameters
    
    if len(params)%2 != 0: # if the number of parameters is not even
        print('The number of parameters must be even!')
        return 
    
    state = QAOA_state(params)
    
    # compute cost function
    # note: should return a negative --> optimization finds minima
    cost = np.dot(state.conjugate().T, np.dot(H, state))[0][0].real

    return cost

# put bounds on 2p = 2(2/N) = N parameters.
# note: p = N/2, but for each layer, p, there are two params
lower_bounds = np.empty(N, dtype=np.float)
lower_bounds.fill(-m.pi)
upper_bounds = np.empty(N, dtype=np.float)
upper_bounds.fill( m.pi)

# initial guess
params0 = np.empty(N, dtype=np.float)
params0.fill(1)
bounds = Bounds(lower_bounds, upper_bounds)

def optimize_overlap():
    
    print_Energy_CostFunction = False
    # note: set verbose to 3 to see full progress
    # set verbose to 0 to see nothing
    
    # note: minimize using trust-constr with bounds is NOT 
    #       as good as minimize using Nelder-Mead without bounds
    
    # optimize with OVERLAP
    #res_overlap = minimize(QAOA_overlap, params0, method='trust-constr', bounds=bounds,
    #                    options={'verbose': 0, 'xtol': 1e-08, 'gtol': 1e-08, 'barrier_tol': 1e-08})
    
    res_overlap = minimize(QAOA_overlap, params0, method='Nelder-Mead', 
                                                                    options={'adaptive': True, 
                                                                                 'disp': None, 
                                                                                 'maxfev': None,
                                                                                 'return_all': False, 
                                                                                 'xatol': 1e-5, 
                                                                                 'fatol': 1e-5})
    
    
    if print_Overlap:
        print('\n')
        print('============== Overlap ===================================')
        print('Overlap = ', -QAOA_overlap(res_overlap.x))
        print('\n')
    if print_Energy_Overlap:
        print('============== Energy (Overlap) ==========================')
        # just to check, find the energy of the overlap-optimized state:
        energy_overlap = np.dot(QAOA_state(res_overlap.x).conjugate().T, 
                        np.dot(H, QAOA_state(res_overlap.x)))[0][0].real
        print('E_QAOA = ', energy_overlap)
        print('\n')
    if print_Optimized_Angles_Overlap:
        print('============== Optimized Angles (Overlap) ================')
        print('[Gamma, Beta] = \n', res_overlap.x)
        print('\n')
    if print_Optimized_State_Overlap:
        print('============== Optimized State (Overlap) =================')
        print(QAOA_state(res_overlap.x))
        print('\n')
    

def optimize_Energy():
    # optimize with Cost Function (Energy)
    
    # note: set verbose to 3 to see full progress
    # set verbose to 0 to see nothing
    
    print_Energy_Overlap= False
    
    # note: minimize using trust-constr with bounds is NOT 
    #       as good as minimize using Nelder-Mead without bounds
    
    #res_cost_function = minimize(QAOA_cost_function, params0, method='trust-constr', bounds=bounds,
    #                    options={'verbose': 0, 'xtol': 1e-08, 'gtol': 1e-08, 'barrier_tol': 1e-08})
    
    res_cost_function = minimize(QAOA_cost_function, params0, method='Nelder-Mead', 
                                                                    options={'adaptive': True, 
                                                                                 'disp': None, 
                                                                                 'maxfev': None,
                                                                                 'return_all': False, 
                                                                                 'xatol': 1e-5, 
                                                                                 'fatol': 1e-5})
    
    if print_Energy_CostFunction:
        print('\n')
        print('============== Energy (Cost Fn) ==========================')
        # just to check, find the energy of the overlap-optimized state:
        energy_costfunction = np.dot(QAOA_state(res_cost_function.x).conjugate().T, 
                        np.dot(H, QAOA_state(res_cost_function.x)))[0][0].real
        
        print('E_QAOA = ', energy_costfunction)
        print('\n')
    if print_Overlap:
        print('============== Overlap ===================================')
        print('Overlap = ', -QAOA_overlap(res_cost_function.x))
        print('\n')
    if print_Optimized_Angles_CostFn:
        print('============== Optimized Angles (CostFn) ================')
        print('[Gamma, Beta] = \n', res_cost_function.x)
        print('\n')
    if print_Optimized_State_CostFn:
        print('============== Optimized State (CostFn) =================')
        print(QAOA_state(res_cost_function.x))
        print('\n')


    
def print_out():
    # printing business...
    if print_g:
        print('============== g =========================================')
        print('g = ', g)
        print('\n')
    if print_Hamiltonian:
        print('============== The Hamiltonian ===========================')
        print(H)
        print('\n')
    if print_Energy_Spectrum:
        print('============== Energy Spectrum ===========================')
        print(e)
        print('\n')
    if print_Matrix_of_Eigenstates:
        print('============== Matrix of Eigenstates =====================')
        print(v)
        print('\n')
    if print_Ground_State_Wfn:
        print('============== Ground State Wfn ==========================')
        print(ground_state)
        print('\n')
    if print_Ground_State_Energy:
        print('============== Ground State Energy =======================')
        print('E_0    = ', ground_state_energy)
        print('\n')

print_out()
optimize_overlap() # using the overlap method
# optimize_Energy()  # using the energy cost function method

# 2. QAOA with non-constant $g$ -- no translational invariance

Same thing, except that now the Hamiltonian is different. $g$ is now an $N$-dimensional array. We can recylce 99% of the previous code and see if we can actually target the ground state in this case, using an QAOA-like ansatz. 

The problem here, of course, is that we no longer have a fixed pair $(\gamma_i,\beta_i)$ per layer. Instead, each layer will contain $2N$ parameters: $N$ values for $\beta_i$ and $N$ values for $\gamma_i$. 

The SciPost paper conjectured that we can still target the ground state using $N/2$ layers of variations. This means our protocol will involve a total of $N/2 * 2N = N^2$ parameters. 

Let's test if this conjecture holds for $N \sim 10$.

In [ ]:
# QAOA for non-constant g
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import expm
import math as m
from scipy.optimize import Bounds, minimize


# Parameters
N = 4 # number of qubits
g = 2*np.random.random_sample((N,)) # N values of g for N sites
p = N//2 # number of layers


#g = np.empty(N, dtype=np.float) # test if constant g still works
#g.fill(2)

print_Hamiltonian = False
print_Energy_Spectrum = False
print_Matrix_of_Eigenstates = False
print_Ground_State_Energy = True
print_Ground_State_Wfn = False
print_Optimized_Angles_Overlap = True
print_Optimized_State_Overlap = False
print_Overlap = True
print_Energy_Overlap = True
print_g = True
print_Energy_CostFunction = True
print_Optimized_Angles_CostFn = True
print_Optimized_State_CostFn = False

Sx = np.array( [[0,1],[1,0]] )
Sz = np.array( [[1,0],[0,-1]] )
Id = np.array( [[1,0],[0,1]] )

def d2b(i,N):
    b = np.array( [ (i//2**(N-1-j))%2 for j in range(N) ] )
    return b
def b2d(b):
    N = len(b)
    d = np.sum( [ b[i]*2**(N-1-i) for i in range(N) ] )
    return d
def gX(N):
    '''
    returns the gX piece of the Hamiltonian
    '''
    gX = np.zeros( (2**N,2**N), dtype = float) # Create matrix of all zeros
    # Add Sx terms one by one, from left to right
    for i in range(N):
        operators = [Id]*i + [np.dot(g[i],Sx)] + [Id]*(N-1-i) # A list of all the operators in this term # <- Change this line
        term = operators[0]
        for op in operators[1:]: # iterate through all elements after the first one, which was used to start the list
            term = np.kron(term, op)
        # Add result for this term to the matrix
        gX += term
    return gX

def ZZ_v3(N):
    ZZ_diag = np.zeros(2**N, dtype = int)
    for col in range(2**N):
        b = d2b(col,N)
        val = 0
        for j in range(N-1):
            val += (-1)**(b[j] + b[j+1])
        ZZ_diag[col] = val
    ZZ = np.diag(ZZ_diag)
    return ZZ

gX = gX(N) 
ZZ = ZZ_v3(N)

def get_H():
    return -ZZ - gX


H = get_H() # get the Hamiltonian
e, v = np.linalg.eigh(H)  # get pairs of eigenvectors and eigenvalues
inds = np.argsort(e) # use this to sort the energy spectrum 
e = e[inds]
v = v[:, inds]

ground_state = v.T[0]/np.linalg.norm(v.T[0])
ground_state_energy = e[0]

# print(H)
#print(ground_state_energy)
# print(ground_state)

def Plus():
    state = np.array([[1,1]])    
    for i in range(N-1):
        state = np.kron(np.array([[1,1]]), state)
    return state.T

# TO DO: modify the existing ExpGB and so on... so that 
# each layer contains 2N parameter values

# plan: keep the QAOA_state function
#       modify the ExpGB so that instead of taking in a single pair gamma, beta
#       it will take in an array [gamma, beta]
#       essentially, what's going on happen is 
#       instead of looking like exp(i beta H_1)
#       it'll look like exp(i \sum (beta_i Z_i Z_{i+1}))
#       ==> basically, need to generate a new "ZZ" and "gX" here
#       ==> need to bring in ZZ, gX functions and modify them

def gX_gamma(gamma):
    gX_gamma = np.zeros( (2**N,2**N), dtype = float) # Create matrix of all zeros
    for i in range(N):
        operators = [Id]*i + [np.dot(gamma[i]*g[i],Sx)] + [Id]*(N-1-i) 
        term = operators[0]
        for op in operators[1:]: 
            term = np.kron(term, op)
        gX_gamma += term
    return gX_gamma

def ZZ_beta(beta):
    ZZ_beta = np.zeros( (2**N,2**N) , dtype = float) # Create matrix of all zeros
    for i in range(N-1):
        operators = [Id]*i + [np.dot(beta[i],Sz),Sz] + [Id]*(N-2-i)
        term = operators[0]
        for op in operators[1:]: 
            term = np.kron(term, op)
        ZZ_beta += term
    return ZZ_beta


def ExpGB(layer_params):
    beta  = layer_params[:len(layer_params)//2]
    gamma = layer_params[len(layer_params)//2:]
   
    return expm(-(1j)*gX_gamma(gamma))@expm(-(1j)*ZZ_beta(beta))


def QAOA_state(params):
    # there are N^2 elements in params
    
    if len(params)%2 != 0: # if the number of parameters is not even
        print('The number of parameters must be even!')
        return 
    
    # initialize state in +
    QAOA_state = Plus()/np.linalg.norm(Plus())
    
    # split list of params
    # N//2 is the number of layers
    param_layers = np.split(params, N//2)
    
    # create QAOA ansatz
    # each layer is a 2N-element array
    
    for i in range(p):  
        QAOA_state = np.dot(ExpGB(param_layers[i]), QAOA_state)
    #print(param_layers)
    return QAOA_state    

def QAOA_overlap(params):
    state = QAOA_state(params)
    # compute overlap
    # note: should return a negative --> optimization finds minima
    overlap = -np.abs( np.vdot(ground_state, state) )**2
    return overlap

def QAOA_cost_function(params):
    # beta is an array of p = N/2 parameters
    # gamma is an array of p = N/2 parameters
    
    if len(params)%2 != 0: # if the number of parameters is not even
        print('The number of parameters must be even!')
        return 
    
    state = QAOA_state(params)
    
    # compute cost function
    # note: should return a negative --> optimization finds minima
    cost = np.dot(state.conjugate().T, np.dot(H, state))[0][0].real

    return cost

#############################################################

# put bounds on N^2 parameters.
# note: p = N/2, but for each layer, p, there are two params
lower_bounds = np.empty(2*N*p, dtype=np.float)
lower_bounds.fill(0)
upper_bounds = np.empty(2*N*p, dtype=np.float)
upper_bounds.fill( m.pi/2)

# initial guess
params0 = np.empty(2*N*p, dtype=np.float)
params0.fill(1)
bounds = Bounds(lower_bounds, upper_bounds)

def optimize_overlap():
    
    print_Energy_CostFunction = False
    # note: set verbose to 3 to see full progress
    # set verbose to 0 to see nothing
    
    # note: minimize using trust-constr with bounds is NOT 
    #       as good as minimize using Nelder-Mead without bounds
    
    # optimize with OVERLAP
    #res_overlap = minimize(QAOA_overlap, params0, method='trust-constr', bounds=bounds,
    #                    options={'verbose': 0, 'xtol': 1e-08, 'gtol': 1e-08, 'barrier_tol': 1e-08})
    
    res_overlap = minimize(QAOA_overlap, params0, method='Nelder-Mead', 
                                                                    options={'adaptive': True, 
                                                                                 'disp': None, 
                                                                                 'maxfev': None,
                                                                                 'return_all': False, 
                                                                                 'xatol': 1e-5, 
                                                                                 'fatol': 1e-5})
    
    
    if print_Overlap:
        print('\n')
        print('============== Overlap ===================================')
        print('Overlap = ', -QAOA_overlap(res_overlap.x))
        print('\n')
    if print_Energy_Overlap:
        print('============== Energy (Overlap) ==========================')
        # just to check, find the energy of the overlap-optimized state:
        energy_overlap = np.dot(QAOA_state(res_overlap.x).conjugate().T, 
                        np.dot(H, QAOA_state(res_overlap.x)))[0][0].real
        print('E_QAOA = ', energy_overlap)
        print('\n')
    if print_Optimized_Angles_Overlap:
        print('============== Optimized Angles (Overlap) ================')
        print('[Gamma, Beta] = \n', res_overlap.x)
        print('\n')
    if print_Optimized_State_Overlap:
        print('============== Optimized State (Overlap) =================')
        print(QAOA_state(res_overlap.x))
        print('\n')
    

def optimize_Energy():
    # optimize with Cost Function (Energy)
    
    # note: set verbose to 3 to see full progress
    # set verbose to 0 to see nothing
    
    print_Energy_Overlap= False
    
    # note: minimize using trust-constr with bounds is NOT 
    #       as good as minimize using Nelder-Mead without bounds
    
    #res_cost_function = minimize(QAOA_cost_function, params0, method='trust-constr', bounds=bounds,
    #                    options={'verbose': 0, 'xtol': 1e-08, 'gtol': 1e-08, 'barrier_tol': 1e-08})
    
    res_cost_function = minimize(QAOA_cost_function, params0, method='Nelder-Mead', 
                                                                    options={'adaptive': True, 
                                                                                 'disp': None, 
                                                                                 'maxfev': None,
                                                                                 'return_all': False, 
                                                                                 'xatol': 1e-5, 
                                                                                 'fatol': 1e-5})
    
    if print_Energy_CostFunction:
        print('\n')
        print('============== Energy (Cost Fn) ==========================')
        # just to check, find the energy of the overlap-optimized state:
        energy_costfunction = np.dot(QAOA_state(res_cost_function.x).conjugate().T, 
                        np.dot(H, QAOA_state(res_cost_function.x)))[0][0].real
        
        print('E_QAOA = ', energy_costfunction)
        print('\n')
    if print_Overlap:
        print('============== Overlap ===================================')
        print('Overlap = ', -QAOA_overlap(res_cost_function.x))
        print('\n')
    if print_Optimized_Angles_CostFn:
        print('============== Optimized Angles (CostFn) ================')
        print('[Gamma, Beta] = \n', res_cost_function.x)
        print('\n')
    if print_Optimized_State_CostFn:
        print('============== Optimized State (CostFn) =================')
        print(QAOA_state(res_cost_function.x))
        print('\n')


    
def print_out():
    # printing business...
    if print_g:
        print('============== g =========================================')
        print('g = ', g)
        print('\n')
    if print_Hamiltonian:
        print('============== The Hamiltonian ===========================')
        print(H)
        print('\n')
    if print_Energy_Spectrum:
        print('============== Energy Spectrum ===========================')
        print(e)
        print('\n')
    if print_Matrix_of_Eigenstates:
        print('============== Matrix of Eigenstates =====================')
        print(v)
        print('\n')
    if print_Ground_State_Wfn:
        print('============== Ground State Wfn ==========================')
        print(ground_state)
        print('\n')
    if print_Ground_State_Energy:
        print('============== Ground State Energy =======================')
        print('E_0    = ', ground_state_energy)

print_out()
optimize_overlap() # using the overlap method
#optimize_Energy()  # using the energy cost function method

### Now, let's see what happens when we reduce the number of layers by $k$.

That is, the number of layers is $ p = N/2 - k$. It turns out the the fidelity, while very high, is not perfect to machine precision. 

In [ ]:
%%time


# QAOA for non-constant g
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import expm
import math as m
from scipy.optimize import Bounds, minimize, LinearConstraint




# Parameters
N = 10 # number of qubits
g = 2*np.random.random_sample((N,)) # N values of g for N sites
p = N//2 # number of layers


#g = np.empty(N, dtype=np.float) # test if constant g still works
#g.fill(2)

print_Hamiltonian = False
print_Energy_Spectrum = False
print_Matrix_of_Eigenstates = False
print_Ground_State_Energy = True
print_Ground_State_Wfn = False
print_Optimized_Angles_Overlap = True
print_Optimized_State_Overlap = False
print_Overlap = True
print_Energy_Overlap = True
print_g = True
print_Energy_CostFunction = True
print_Optimized_Angles_CostFn = True
print_Optimized_State_CostFn = False

Sx = np.array( [[0,1],[1,0]] )
Sz = np.array( [[1,0],[0,-1]] )
Id = np.array( [[1,0],[0,1]] )

def d2b(i,N):
    b = np.array( [ (i//2**(N-1-j))%2 for j in range(N) ] )
    return b
def b2d(b):
    N = len(b)
    d = np.sum( [ b[i]*2**(N-1-i) for i in range(N) ] )
    return d
def gX(N):
    '''
    returns the gX piece of the Hamiltonian
    '''
    gX = np.zeros( (2**N,2**N), dtype = float) # Create matrix of all zeros
    # Add Sx terms one by one, from left to right
    for i in range(N):
        operators = [Id]*i + [np.dot(g[i],Sx)] + [Id]*(N-1-i) # A list of all the operators in this term # <- Change this line
        term = operators[0]
        for op in operators[1:]: # iterate through all elements after the first one, which was used to start the list
            term = np.kron(term, op)
        # Add result for this term to the matrix
        gX += term
    return gX

def ZZ_v3(N):
    ZZ_diag = np.zeros(2**N, dtype = int)
    for col in range(2**N):
        b = d2b(col,N)
        val = 0
        for j in range(N-1):
            val += (-1)**(b[j] + b[j+1])
        ZZ_diag[col] = val
    ZZ = np.diag(ZZ_diag)
    return ZZ

gX = gX(N) 
ZZ = ZZ_v3(N)

def get_H():
    return -ZZ - gX


H = get_H() # get the Hamiltonian
e, v = np.linalg.eigh(H)  # get pairs of eigenvectors and eigenvalues
inds = np.argsort(e) # use this to sort the energy spectrum 
e = e[inds]
v = v[:, inds]

ground_state = v.T[0]/np.linalg.norm(v.T[0])
ground_state_energy = e[0]

# print(H)
#print(ground_state_energy)
# print(ground_state)

def Plus():
    state = np.array([[1,1]])    
    for i in range(N-1):
        state = np.kron(np.array([[1,1]]), state)
    return state.T

# TO DO: modify the existing ExpGB and so on... so that 
# each layer contains 2N parameter values

# plan: keep the QAOA_state function
#       modify the ExpGB so that instead of taking in a single pair gamma, beta
#       it will take in an array [gamma, beta]
#       essentially, what's going on happen is 
#       instead of looking like exp(i beta H_1)
#       it'll look like exp(i \sum (beta_i Z_i Z_{i+1}))
#       ==> basically, need to generate a new "ZZ" and "gX" here
#       ==> need to bring in ZZ, gX functions and modify them

def gX_gamma(gamma):
    gX_gamma = np.zeros( (2**N,2**N), dtype = float) # Create matrix of all zeros
    for i in range(N):
        operators = [Id]*i + [np.dot(gamma[i]*g[i],Sx)] + [Id]*(N-1-i) 
        term = operators[0]
        for op in operators[1:]: 
            term = np.kron(term, op)
        gX_gamma += term
    return gX_gamma

def ZZ_beta(beta):
    ZZ_beta = np.zeros( (2**N,2**N) , dtype = float) # Create matrix of all zeros
    for i in range(N-1):
        operators = [Id]*i + [np.dot(beta[i],Sz),Sz] + [Id]*(N-2-i)
        term = operators[0]
        for op in operators[1:]: 
            term = np.kron(term, op)
        ZZ_beta += term
    return ZZ_beta


def ExpGB(layer_params):
    beta  = layer_params[:len(layer_params)//2]
    gamma = layer_params[len(layer_params)//2:]
   
    return expm(-(1j)*gX_gamma(gamma))@expm(-(1j)*ZZ_beta(beta))


def QAOA_state(params):
    # there are N^2 elements in params
    
    if len(params)%2 != 0: # if the number of parameters is not even
        print('The number of parameters must be even!')
        return 
    
    # initialize state in +
    QAOA_state = Plus()/np.linalg.norm(Plus())
    
    # split list of params
    # p = N//2 - k is the number of layers
    param_layers = np.split(params, p)
    
    # create QAOA ansatz
    # each layer is a 2N-element array
    
    for i in range(p):  
        QAOA_state = np.dot(ExpGB(param_layers[i]), QAOA_state)
    #print(param_layers)
    return QAOA_state    

def QAOA_overlap(params):
    state = QAOA_state(params)
    # compute overlap
    # note: should return a negative --> optimization finds minima
    overlap = -np.abs( np.vdot(ground_state, state) )**2
    return overlap

def QAOA_cost_function(params):
    # beta is an array of p = N/2 parameters
    # gamma is an array of p = N/2 parameters
    
    if len(params)%2 != 0: # if the number of parameters is not even
        print('The number of parameters must be even!')
        return 
    
    state = QAOA_state(params)
    
    # compute cost function
    # note: should return a negative --> optimization finds minima
    cost = np.dot(state.conjugate().T, np.dot(H, state))[0][0].real

    return cost

#############################################################

# put bounds on N^2 parameters.
# note: p = N/2, but for each layer, p, there are two params
lower_bounds = np.empty(2*N*p, dtype=np.float)
lower_bounds.fill(0)
upper_bounds = np.empty(2*N*p, dtype=np.float)
upper_bounds.fill( m.pi/2)

# initial guess
params0 = np.empty(2*N*p, dtype=np.float)
params0.fill(1)
bounds = Bounds(lower_bounds, upper_bounds)

def optimize_overlap():
    
    print_Energy_CostFunction = False
    # note: set verbose to 3 to see full progress
    # set verbose to 0 to see nothing
    
    # note: minimize using trust-constr with bounds is NOT 
    #       as good as minimize using Nelder-Mead without bounds
    
    # optimize with OVERLAP
    #res_overlap = minimize(QAOA_overlap, params0, method='trust-constr', bounds=bounds,
    #                    options={'verbose': 0, 'xtol': 1e-08, 'gtol': 1e-08, 'barrier_tol': 1e-08})
    
    #res_overlap = minimize(QAOA_overlap, params0, method='Nelder-Mead', 
    #                                                                options={'adaptive': True, 
    #                                                                             'disp': None, 
    #                                                                             'maxfev': None,
    #                                                                             'return_all': False, 
    #                                                                             'xatol': 1e-5, 
    #                                                                             'fatol': 1e-5})
    
    I = np.zeros((2*N*p, 2*N*p), float)
    np.fill_diagonal(I,1)
    constraints = LinearConstraint(I,lower_bounds,upper_bounds)
    
    res_overlap = minimize(QAOA_overlap, params0, method='COBYLA', constraints = constraints, tol= 1e-3, 
                           options={'rhobeg': 1.0, 'maxiter': 1000, 'disp': True, 'catol': 0.0002})
    
    
    if print_Overlap:
        print('\n')
        print('============== Overlap ===================================')
        print('Overlap = ', -QAOA_overlap(res_overlap.x))
        print('\n')
    if print_Energy_Overlap:
        print('============== Energy (Overlap) ==========================')
        # just to check, find the energy of the overlap-optimized state:
        energy_overlap = np.dot(QAOA_state(res_overlap.x).conjugate().T, 
                        np.dot(H, QAOA_state(res_overlap.x)))[0][0].real
        print('E_QAOA = ', energy_overlap)
        print('\n')
    if print_Optimized_Angles_Overlap:
        print('============== Optimized Angles (Overlap) ================')
        print('[Gamma, Beta] = \n', res_overlap.x)
        print('\n')
    if print_Optimized_State_Overlap:
        print('============== Optimized State (Overlap) =================')
        print(QAOA_state(res_overlap.x))
        print('\n')
    

def optimize_Energy():
    # optimize with Cost Function (Energy)
    
    # note: set verbose to 3 to see full progress
    # set verbose to 0 to see nothing
    
    print_Energy_Overlap= False
    
    # note: minimize using trust-constr with bounds is NOT 
    #       as good as minimize using Nelder-Mead without bounds
    
    #res_cost_function = minimize(QAOA_cost_function, params0, method='trust-constr', bounds=bounds,
    #                    options={'verbose': 0, 'xtol': 1e-08, 'gtol': 1e-08, 'barrier_tol': 1e-08})
    
    res_cost_function = minimize(QAOA_cost_function, params0, method='Nelder-Mead', 
                                                                    options={'adaptive': True, 
                                                                                 'disp': None, 
                                                                                 'maxfev': None,
                                                                                 'return_all': False, 
                                                                                 'xatol': 1e-5, 
                                                                                 'fatol': 1e-5})
    
    if print_Energy_CostFunction:
        print('\n')
        print('============== Energy (Cost Fn) ==========================')
        # just to check, find the energy of the overlap-optimized state:
        energy_costfunction = np.dot(QAOA_state(res_cost_function.x).conjugate().T, 
                        np.dot(H, QAOA_state(res_cost_function.x)))[0][0].real
        
        print('E_QAOA = ', energy_costfunction)
        print('\n')
    if print_Overlap:
        print('============== Overlap ===================================')
        print('Overlap = ', -QAOA_overlap(res_cost_function.x))
        print('\n')
    if print_Optimized_Angles_CostFn:
        print('============== Optimized Angles (CostFn) ================')
        print('[Gamma, Beta] = \n', res_cost_function.x)
        print('\n')
    if print_Optimized_State_CostFn:
        print('============== Optimized State (CostFn) =================')
        print(QAOA_state(res_cost_function.x))
        print('\n')

    
def print_out():
    # printing business...
    if print_g:
        print('============== g =========================================')
        print('g = ', g)
        print('\n')
    if print_Hamiltonian:
        print('============== The Hamiltonian ===========================')
        print(H)
        print('\n')
    if print_Energy_Spectrum:
        print('============== Energy Spectrum ===========================')
        print(e)
        print('\n')
    if print_Matrix_of_Eigenstates:
        print('============== Matrix of Eigenstates =====================')
        print(v)
        print('\n')
    if print_Ground_State_Wfn:
        print('============== Ground State Wfn ==========================')
        print(ground_state)
        print('\n')
    if print_Ground_State_Energy:
        print('============== Ground State Energy =======================')
        print('E_0    = ', ground_state_energy)

print_out()
optimize_overlap() # using the overlap method
#optimize_Energy()  # using the energy cost function method

============== g =========================================
g =  [0.03637692 1.78813028 0.94413777 0.26840726 0.92133876 0.12131155
 0.28042323 1.48834581 1.31801806 1.70497107]


============== Ground State Energy =======================
E_0    =  -12.414264477454303


# 2. QAOA with constant $g$ & The Jordan-Wigner Transform